In [2]:
import torch
import numpy as np
from stable_baselines3 import SAC

import os, shutil, sys
sys.path.append(os.path.abspath('./env'))
sys.path.append(os.path.abspath('./common'))

from env.env_move_sector_v2 import HumanMoveSectorActionV2
from env.env_move_ray_v3  import HumanMoveRayActionV3

import common.mlflow_sb3_helper as mlf
import common.tensorboard_sb3_helper as tnb

USE_MLFLOW = False


from pytz import timezone
from datetime import datetime

TZ = timezone('Europe/Moscow')

activations = {'ReLU':torch.nn.ReLU, 'Tanh':torch.nn.Tanh}
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
exp_params = {
    'env_name': '',
    'algorithm_name': '',
    'exp_id': 0,
    'exp_name': f'exp_{datetime.now(TZ).strftime("%d%m_%H%M%S")}',
    'seed': 21,
    'net': {
        'activation': 'Tanh',
        'pi': [512, 256, 128],
        'qf': [512, 256, 128],
    },
    'training': {
        'iteration_count': 6,
        'episode_count': 39000,
        'policy': 'MlpPolicy',
        'learning_rate': 0.001,
        'buffer_size': 1500000,
        'learning_starts': 100,
        'batch_size': 1024,
        'tau': 0.005,
        'gamma': 0.99,
        'verbose': 0,
        'device': device,
    },
    'validation': {
        'validate_agent_every_n_eps': 20000,
        'log_interval': 10
    },
}

option_var ={
    'SAC 100_0':{'finish_dist':100, 'start_dist':110, 'delta_phi':0 },
    'SAC 80_0':{'finish_dist':80, 'start_dist':110, 'delta_phi':0 },
    'SAC 60_0':{'finish_dist':60, 'start_dist':110, 'delta_phi':0 },
    #'SAC 40_0':{'finish_dist':40, 'start_dist':110, 'delta_phi':0 },
    #'SAC 20_0':{'finish_dist':20, 'start_dist':110, 'delta_phi':0 },
    #'SAC 10_0':{'finish_dist':10, 'start_dist':110, 'delta_phi':0 },
    #'SAC 10_10':{'finish_dist':10, 'start_dist':110, 'delta_phi':10 },
    #'SAC 10_20':{'finish_dist':10, 'start_dist':110, 'delta_phi':20 },
    #'SAC 10_30':{'finish_dist':10, 'start_dist':110, 'delta_phi':30 },
    #'SAC 10_40':{'finish_dist':10, 'start_dist':110, 'delta_phi':40 },
}

option_var2 ={
    'SAC 100_0':{'finish_dist':100, 'start_dist':110, 'delta_phi':0 },
    'SAC 80_0':{'finish_dist':80, 'start_dist':110, 'delta_phi':0 },
    'SAC 80_20':{'finish_dist':80, 'start_dist':110, 'delta_phi':20 },
    'SAC 80_45':{'finish_dist':80, 'start_dist':110, 'delta_phi':45 },
    'SAC 80_90':{'finish_dist':80, 'start_dist':110, 'delta_phi':90 },
    'SAC 80_135':{'finish_dist':80, 'start_dist':110, 'delta_phi':135 },
    'SAC 80_180':{'finish_dist':80, 'start_dist':110, 'delta_phi':180 },
    'SAC 60_180':{'finish_dist':60, 'start_dist':110, 'delta_phi':180 },
    'SAC 40_180':{'finish_dist':40, 'start_dist':110, 'delta_phi':180 },
    'SAC 30_180':{'finish_dist':30, 'start_dist':110, 'delta_phi':180 },
    'SAC 20_180':{'finish_dist':20, 'start_dist':110, 'delta_phi':180 },
    'SAC 10_180':{'finish_dist':10, 'start_dist':110, 'delta_phi':180 },
}



In [4]:

policy_kwargs = dict(activation_fn = activations[exp_params['net']['activation']],
                     net_arch = dict(pi=exp_params['net']['pi'], qf=exp_params['net']['qf']))




In [5]:
log_helper = {}
log_path = {}

if USE_MLFLOW:
    log_helper = mlf.MLflowServerHelper("http://192.168.0.206:2670", False)
else:
    log_helper = tnb.TensorboardHelper("",False)

In [6]:
env = HumanMoveRayActionV3(target_point_rand=False, object_ignore=True)
env_render = HumanMoveRayActionV3(target_point_rand=False, object_ignore=True, render_mode='rgb_array')

In [7]:
#!!!!! Новый эксперимент
exp_params['env_name'] = env.name()
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = log_helper.new_experiment(exp_name)

In [ ]:
#!!!!! Дорасчет старого эксперимента
exp_params['env_name'] = env.name()
experiment_id = 534
exp_name = log_helper.get_experiment(experiment_id)
print(exp_name)

In [8]:
exp_params['exp_id'] = experiment_id
exp_params['algorithm_name'] = "SAC"
exp_params['env_name'] = env.name()

b_first = True
model = None

for name, option in option_var2.items():

    print(name)

    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))
    exp_params['options'] = option


    env.set_options(option)
    env_render.set_options(option)

    if b_first == True:
        b_first = False
        model = SAC(  exp_params['training']['policy'], 
                    env,
                    policy_kwargs=policy_kwargs,
                    learning_rate=exp_params['training']['learning_rate'],
                    buffer_size=exp_params['training']['buffer_size'],
                    learning_starts=exp_params['training']['learning_starts'],
                    batch_size=exp_params['training']['batch_size'],
                    tau=exp_params['training']['tau'],
                    gamma=exp_params['training']['gamma'],
                    verbose=exp_params['training']['verbose'],
                    device=device,
                    )
    else:
        print(exp_name)
        model = SAC.load(f'{exp_name}/model.zip', env=env, device=device)


    art_loc, exp_name, run_id = log_helper.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=name,
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'],
                                video_episods=3,
                                video_fps=120   
                                )

    log_path[name] = f'{art_loc}/{run_id}/artifacts/{exp_name}/sb3/model.zip'



SAC 100_0


Output()

Moviepy - Building video env_MoveRay3_IgnoreObst_exp_0201_203654/SAC 100_0//agent.mp4.
Moviepy - Writing video env_MoveRay3_IgnoreObst_exp_0201_203654/SAC 100_0//agent.mp4



Moviepy - Done !
Moviepy - video ready env_MoveRay3_IgnoreObst_exp_0201_203654/SAC 100_0//agent.mp4
SAC 80_0
env_MoveRay3_IgnoreObst_exp_0201_203654/SAC 100_0


Output()

In [ ]:
log_path

In [5]:
experiment_id = 554
log_path = {
 'SAC': f'mlflow-artifacts:/{experiment_id}/14a433f1610a4dc0837b7c163f2df4b4/artifacts/env_MoveSector_IgnoreObst_exp_3010_165101/sb3/model.zip'
 }

In [ ]:
load_path = os.path.join(str(experiment_id))
if  os.path.isdir(load_path) == False:
    os.mkdir(load_path)
load_path

In [ ]:

loadel_models = {}

for model_name, path in log_path.items():
    loadel_models[model_name] =  f'{load_path}/{model_name}/'
    log_helper.load_artifact(path,loadel_models[model_name])

In [8]:
env = HumanMoveSectorActionV2(target_point_rand=False, object_ignore=True)
env_render = HumanMoveSectorActionV2(target_point_rand=False, object_ignore=True, render_mode='rgb_array')

In [ ]:
#!!! Дорасчет в НОВОМ эксперименте
exp_params['env_name'] = env.name()
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = log_helper.new_experiment(exp_name)
exp_params['exp_id'] = experiment_id
print(exp_name)


In [ ]:
#!!! Дорасчет в СТАРОМ эксперименте
exp_name = log_helper.get_experiment(experiment_id)
print(exp_name)

In [ ]:

exp_params['algorithm_name'] = 'SAC'

path = loadel_models['SAC']
print(path)
name = 'SAC'

is_first = True

for i in range(exp_params['training']['iteration_count']):
    
    exp_params['algorithm_name'] = name
    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))

    if is_first:
        model = SAC.load(f'{path}/model.zip', env=env, device=device)
        is_first = False
    else:
        model = SAC.load(f'{exp_name}/model.zip', env=env, device=device)

    art_loc, exp_name, run_id = log_helper.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=f'{name}_{i+5}',
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'],
                                video_episods=3,
                                video_fps=120                                   
                                )

In [10]:

option_var3 ={
    #'SAC 5_110_180':{'finish_dist':5, 'start_dist':110, 'delta_phi':180 },
    #'SAC 5_120_180':{'finish_dist':5, 'start_dist':120, 'delta_phi':180 },
    #'SAC 5_130_180':{'finish_dist':5, 'start_dist':130, 'delta_phi':180 },
    #'SAC 5_140_180':{'finish_dist':5, 'start_dist':140, 'delta_phi':180 },
    #'SAC 5_150_180':{'finish_dist':5, 'start_dist':150, 'delta_phi':180 },
    'SAC 5_160_180':{'finish_dist':5, 'start_dist':160, 'delta_phi':180 },
    'SAC 5_170_180':{'finish_dist':5, 'start_dist':170, 'delta_phi':180 },
    'SAC 5_180_180':{'finish_dist':5, 'start_dist':180, 'delta_phi':180 },
    'SAC 5_190_180':{'finish_dist':5, 'start_dist':190, 'delta_phi':180 },
    'SAC 5_200_180':{'finish_dist':5, 'start_dist':200, 'delta_phi':180 },
    'SAC 5_210_180':{'finish_dist':5, 'start_dist':210, 'delta_phi':180 },
    'SAC 5_220_180':{'finish_dist':5, 'start_dist':220, 'delta_phi':180 },
}


option_var4 ={
    'SAC 10_180_1':{'finish_dist':10, 'start_dist':110, 'delta_phi':180 },
    'SAC 10_180_2':{'finish_dist':10, 'start_dist':110, 'delta_phi':180 },
    'SAC 10_180_3':{'finish_dist':10, 'start_dist':110, 'delta_phi':180 },
    'SAC 10_180_4':{'finish_dist':10, 'start_dist':110, 'delta_phi':180 },
}


In [ ]:
exp_params['algorithm_name'] = 'SAC'

path = loadel_models['SAC']
print(path)

b_first = True
model = None

for name, option in option_var3.items():

    print(name)

    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))
    exp_params['options'] = option


    env.set_options(option)
    env_render.set_options(option)

    if b_first == True:
        b_first = False
        model = SAC.load(f'{path}/model.zip', env=env, device=device)
    else:
        print(exp_name)
        model = SAC.load(f'{exp_name}/model.zip', env=env, device=device)


    art_loc, exp_name, run_id = log_helper.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=name,
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'],
                                video_episods=3,
                                video_fps=120   
                                )

    log_path[name] = f'{art_loc}/{run_id}/artifacts/{exp_name}/sb3/model.zip'